In [1]:
!pip install kaleido cohere openai tiktoken pyts gradio seaborn scikit-learn numpy==1.24.3 pydeseq2==0.4.3 statsmodels==0.14.0 yfinance neo4j mplfinance yahoo_fin openpyxl altair prompt_toolkit

### Configuration

1. Add a new **utility**
```
CREATE (u:utility {market:'', code: '', name: ''})
```
[market]
  * yf.Ticker(code): 'TWSE', 'NASDAQ', 'ACE'
  *

In [2]:
#yf.Ticker('GBTC').history(period='1d')

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import json, warnings, time
import yfinance as yf
import mplfinance as mpf
import pandas as pd
from neo4j import GraphDatabase
import scipy.stats as stats
from prompt_toolkit.application.application import E
import yahoo_fin.stock_info as si
import matplotlib as mpl
from scipy.optimize import linprog
from sklearn.linear_model import LinearRegression
import gradio as gr
import altair as alt

warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter('ignore', FutureWarning)
AURA_CONNECTION_URI = "neo4j+s://6d2f5b5d.databases.neo4j.io"
AURA_USERNAME = "neo4j"
AURA_PASSWORD = "ZzZ6zeBZ1N7fB_UAHezzHY0LajAXj2z7tmI2HwHPWa8" #@param ["ZzZ6zeBZ1N7fB_UAHezzHY0LajAXj2z7tmI2HwHPWa8"]
driver = GraphDatabase.driver(AURA_CONNECTION_URI, auth=(AURA_USERNAME, AURA_PASSWORD))

TWD_option = True
USD_option = True #@param ['True', 'False']
USDT_option = True #@param ['True', 'False']

loan_interest = 0.07
timeperiod = 90

### Initialization

In [4]:
#techinical_index
def sma(close, timeperiod):
  sma = np.empty_like(close)
  for i in range(len(close)):
    if i < timeperiod:
      sma[i] = np.nan
    else:
      sma[i] = np.mean(close[i - timeperiod + 1: i + 1])
  return sma

def stddev(close, timeperiod):
    stddev = np.empty_like(close)
    stddev[:] = np.nan
    for i in range(timeperiod - 1, len(close)):
        window = close[i - timeperiod + 1:i + 1]
        stddev[i] = np.std(window, ddof=0)
    return stddev

def tanh(x):
  x = x
  return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

def bias_degree(index, timeperiod = timeperiod): #index_summary
  for i in index:
    index[i]['MEAN'] = sma(index[i].Close, timeperiod = timeperiod)
    index[i]['STD'] = stddev(index[i].Close, timeperiod = timeperiod)
    index[i]['HCL0'] = np.where(index[i]['Low'] > (index[i]['MEAN'] + 1 * index[i]['STD']), 1, 0)
    index[i]['HCL1'] = np.where(index[i]['Low'] > (index[i]['MEAN'] + 2 * index[i]['STD']), 1, 0)
    index[i]['HCL2'] = np.where(index[i]['Low'] > (index[i]['MEAN'] + 3 * index[i]['STD']), 1, 0)
    index[i]['LCL0'] = np.where(index[i]['High'] < (index[i]['MEAN'] - 1 * index[i]['STD']), 1, 0)
    index[i]['LCL1'] = np.where(index[i]['High'] < (index[i]['MEAN'] - 2 * index[i]['STD']), 1, 0)
    index[i]['LCL2'] = np.where(index[i]['High'] < (index[i]['MEAN'] - 3 * index[i]['STD']), 1, 0)
    index[i]['STATE'] = index[i]['HCL0'] + index[i]['HCL1'] + index[i]['HCL2'] - index[i]['LCL0'] - index[i]['LCL1'] - index[i]['LCL2']
    index[i].index = index[i].index.date
  df = index[list(index.keys())[-1]].copy()
  for i in index:
    df[i] = index[i].STATE
  return df

def calc_beta(df):
    np_array = df.values
    s = np_array[:,0]
    m = np_array[:,1]
    covariance = np.cov(s,m)
    beta = covariance[0,1]/covariance[1,1]
    return beta

def rolling_apply(df, period, func, min_periods=None):
    if min_periods is None:
        min_periods = period
    result = pd.Series(np.nan, index=df.index)
    for i in range(1, len(df)+1):
        sub_df = df.iloc[max(i-period, 0):i,:]
        if len(sub_df) >= min_periods:
            idx = sub_df.index[-1]
            result[idx] = func(sub_df)
    return result

#financial indicator
def forex_risk(x):
  y = x[-1]
  MEAN, STD = np.mean(x), np.std(x)
  LCL, HCL = (MEAN - STD * 2), (MEAN + STD * 2)
  _, double_p = stats.ttest_ind(np.random.normal(loc=MEAN, scale=STD * MEAN, size=len(x)), x * (y / MEAN), equal_var = False)
  if np.mean(x) > np.mean(y):
    p = double_p/2.
  else:
    p = 1.0 - double_p/2.
  return -(HCL / MEAN - 1) * round(p, 2)

def Economic_Status():
  market, forex, risk, level = {}, {}, {}, {'x': list(range(timeperiod))}
  for i in ['^HSI', '^HSCE', '000001.SS', '399001.SZ', '^N225', '^DJI', '^GSPC', '^IXIC', '^KS11', '^TWII']:
    market[i] = yf.Ticker(i).history(period='6y')
  for i in ['USDTWD=X', 'USDCNY=X', 'USDJPY=X', 'USDEUR=X', 'USDHKD=X', 'USDKRW=X']:
    forex[i] = yf.Ticker(i).history(period='6y')
    risk[i] = forex_risk(forex[i].Close)
    forex_std = list(forex[i].Close[-timeperiod:] / np.std(forex[i].Close))
    level[i.replace('=X', '').replace('USD', '')] = forex_std / np.mean(forex_std)

  market_df = bias_degree(market).iloc[:, 16:].copy()
  forex_df = bias_degree(forex)
  for i in forex_df.iloc[:, 16:]:
    market_df[i] = forex_df[i]
  for i in market_df:
    market_df[i] = market_df[i].ffill()
  temp_df = market_df.iloc[: -1, :].dropna().copy()
  col = ['^DJI', '^GSPC', '^IXIC', '^TWII']
  X, y, pred_X = np.array(temp_df.iloc[: -1, :]), np.array(temp_df.loc[:, col][1: ]), np.array(temp_df.iloc[-1:, :])
  model = LinearRegression().fit(X, y)
  summary = pd.DataFrame(np.array([model.predict(pred_X)[0]]), columns=col).apply(tanh)
  summary['indexes'] = [list(temp_df.index)[-1]]
  summary = summary.set_index('indexes')
  summary['^TW'] = summary['^TWII']
  summary['^US'] = (summary['^DJI'] + summary['^GSPC'] + summary['^IXIC'])/3

  #history level of each forex (gradio)
  data_melted = pd.DataFrame(level).melt('x', var_name='line', value_name='y')
  chart = alt.Chart(data_melted).mark_line().encode(x='x', y=alt.Y('y', scale=alt.Scale(domain=[0.95, 1.05])), color='line').properties(height=100)
  return summary.loc[:, ["^TW", "^US"]], forex, risk, chart

summary, forex, risk, chart = Economic_Status()
print(list(forex.keys()))
print(risk)
summary

['USDTWD=X', 'USDCNY=X', 'USDJPY=X', 'USDEUR=X', 'USDHKD=X', 'USDKRW=X']
{'USDTWD=X': -0.0806890264000361, 'USDCNY=X': -0.08456067305292292, 'USDJPY=X': -0.24505836903726852, 'USDEUR=X': -0.10277943925795752, 'USDHKD=X': -0.009078224085758252, 'USDKRW=X': -0.07176536233611974}


,^TW,^US
indexes,,
2024-03-01,0.947499,0.761265


### Update Informations

In [5]:
def update_price_and_eps(df, name, market):
  data = df.history(period='1d')
  update, price = data.index[-1].strftime('%Y-%m-%d'), float(data.Close[-1])
  if market == 'TWSE':
    price /= forex['USDTWD=X'].Close[-1]
  try:
    eps = round(df.income_stmt.loc['Diluted EPS', :].dropna()[0], 2)
  except:
    eps = 0
  price = round(price, 2)
  query = "MATCH (u:utility WHERE u.name='{name}') set u.price='{price}' set u.eps='{eps}' set u.update='{update}'".format(name=utility['u']['name'], price=str(price), eps=str(eps), update=update)
  session.run(query)
  query = "MATCH (u:utility WHERE u.name='{name}') remove u.PE_ratio"
  session.run(query)
  return name

def update_beta_and_pe(name, code, market):
  quote = si.get_quote_table(code)
  if market == 'ACE' or market == 'NYSE':
    df = yf.download(code, period='max')
    beta = rolling_apply(df, timeperiod, calc_beta)[-1]
    df['TVL'] = df['Adj Close'] * 100/ df['Adj Close'][0]
    pe_ratio = list(df['TVL']/100)[-1]
  else:
    beta, pe_ratio = quote['Beta (5Y Monthly)'], quote['PE Ratio (TTM)']
  query = "MATCH (u:utility WHERE u.name='{name}') set u.pe_ratio='{pe_ratio}' set u.beta='{beta}'".format(name=utility['u']['name'], pe_ratio=str(pe_ratio), beta=str(beta))
  session.run(query)
  return name

sharpe_dict = {}
query = "MATCH (u:utility) RETURN u"
with driver.session() as session:
    results = session.run(query).data()
    update = []
    #Stage 1:
    for utility in results:
      name, code, market = utility['u']['name'], utility['u']['code'], utility['u']['market']
      df = yf.Ticker(code)
      update.append(update_price_and_eps(df, name, market))
    print('Price updated', update)

    #Stage 2:
    update = []
    for utility in results:
      name, code, market = utility['u']['name'], utility['u']['code'], utility['u']['market']
      update.append(update_beta_and_pe(name, code, market))
    print('Beta/PE updated', update)

    #Stage 3:
    for utility in results:
      if "sharpo" in utility['u'].keys():
        if '/' in utility['u']['sharpo']:
          sharpe_dict[utility['u']['name']] = 0
        else:
          sharpe_dict[utility['u']['name']] = float(utility['u']['sharpo'])
      else:
        sharpe_dict[utility['u']['name']] = 0
    print('Initialize Sharpo Record')

Price updated ['Micron', 'TSMC', 'ASML', 'ARM', 'NVIDIA', 'MediaTek', 'AMD', 'Intel', 'GBTC', '台達電', '英業達', '鴻海', '廣達', '緯穎', '技嘉', '德州儀器', 'Qualcomm', '中華電', 'Tesla', 'Amazon', 'Microsoft', 'Meta', 'Apple', 'NXP', 'Coinbase', 'Alphabet', 'RIOT', 'MARA', 'HIVE', 'BTC', 'ETH', 'BNB', 'ADA', 'IBM']


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Beta/PE updated ['Micron', 'TSMC', 'ASML', 'ARM', 'NVIDIA', 'MediaTek', 'AMD', 'Intel', 'GBTC', '台達電', '英業達', '鴻海', '廣達', '緯穎', '技嘉', '德州儀器', 'Qualcomm', '中華電', 'Tesla', 'Amazon', 'Microsoft', 'Meta', 'Apple', 'NXP', 'Coinbase', 'Alphabet', 'RIOT', 'MARA', 'HIVE', 'BTC', 'ETH', 'BNB', 'ADA', 'IBM']
Initialize Sharpo Record


In [8]:
def process(option, distribute, driver):
  labels, columns = {}, []
  for cls in ["field", "topic", "product", "utility"]:
    query = "MATCH (n:{cls}) RETURN n".format(cls=cls)
    with driver.session() as session:
      results = session.run(query).data()
      labels[cls] = [i['n']['name'] for i in results]
      columns += labels[cls]
  transition_matrix, columns = np.zeros((len(columns), len(columns))), np.array(columns)

  level_state = np.zeros((len(columns)))
  pe_pair = []
  for utility in results:
    try:
      if utility['n']['PE_ratio'] != 'nan':
        pe_pair.append([utility['n']['name'], utility['n']['PE_ratio']])
    except:
      pass
  pe_pair = np.array(pe_pair)
  base_count = 0
  for name, pe_ratio in pe_pair:
    _, p = stats.ttest_ind(pe_pair[:,1].astype(float), [float(pe_ratio)])
    #if p > 0.05:
    if 1 == 1:
      i = np.where(columns==str(name))[0][0]
      level_state[i] = pe_ratio; base_count += 1
  level_state = np.array(level_state)/ base_count
  absorbing_node = [np.where(columns==cls)[0][0] for cls in labels[option]]
  for i in absorbing_node:
    transition_matrix[i][i] = 1
  query = "MATCH p=()-[:dominate]->() RETURN p"
  with driver.session() as session:
    results = session.run(query).data()
    for vertex in results:
      i, j = np.where(columns==vertex['p'][0]['name'])[0][0], np.where(columns==vertex['p'][2]['name'])[0][0]
      if i not in absorbing_node:
        transition_matrix[i][j] = 1
      if option != "field":
        if vertex['p'][2]['name'] in labels["field"]:
          transition_matrix[j][i] = 1
  for i in range(len(columns)):
    if np.sum(transition_matrix[i]) > 0:
      transition_matrix[i] /= np.sum(transition_matrix[i])
  state = np.zeros((len(columns)))
  cash = 1
  for cls in distribute:
    i = np.where(columns==cls)[0][0]
    state[i] = distribute[cls]
    cash -= state[i]

  res = {'category': [], 'value': []}
  for _ in range(len(columns)):
    state = state.dot(transition_matrix)

  slack = 1 - cash
  for i, j in enumerate(absorbing_node):
    slack -= state[absorbing_node][i]
    print(columns[j], ':', round(state[absorbing_node][i] * 100, 2), '%')
    res['category'].append(columns[j])
    res['value'].append(state[absorbing_node][i] * 100)
  print('Other :', round(slack * 100, 2), '%')

  res['category'].append("其它"); res['value'].append(slack * 100)
  res['category'].append("現金"); res['value'].append(cash * 100)
  for _ in range(len(columns)):
    level_state = level_state.dot(transition_matrix)
  print('\n市場整體本益比:', round(np.sum(level_state * state), 2))
  return res

def estimate(df, distribute):
  beta, sharpe, pe_ratio = 0, 0, 0
  for name in distribute:
    beta += float(df[df['name']==name]['BETA']) * distribute[name]
    sharpe += float(df[df['name']==name]['Sharpe']) * distribute[name]
    pe_ratio += float(df[df['name']==name]['PE_ratio']) * distribute[name]
  return 'Beta係數:' + str(round(beta, 2)) +', Sharpe指標:' + str(round(sharpe, 2)) +', 整體本益狀況:' + str(round(pe_ratio, 2))

def finance_report(Sharpe, option):
  Sharpe_temp = list(Sharpe.values())
  company_names = list(Sharpe.keys())
  table = {'name':[], 'price': [], 'eps': [], 'BETA': [], 'PE_ratio': []}
  query = "MATCH (u:utility) RETURN u"
  with driver.session() as session:
    results = session.run(query).data()
    for utility in results:
      if 'eps' in utility['u'].keys():
        i = np.where(np.array(company_names)==str(utility['u']['name']))[0][0]
        if utility['u']['code'].endswith('.TW'):
          Sharpe_temp[i] -= loan_interest * (1 - risk['USDTWD=X'])
        else:
          Sharpe_temp[i] -= loan_interest
        table['name'].append(utility['u']['name']); table['price'].append(utility['u']['price']); table['eps'].append(utility['u']['eps'])
        table['BETA'].append(utility['u']['beta']); table['PE_ratio'].append(utility['u']['pe_ratio'])
  df = pd.DataFrame(table)
  df['eps'], df['price'] = df['eps'].astype(float), df['price'].astype(float)
  df['投資報酬率(%)/年'] = (df['eps'].astype(float) * 100 / df['price']).round(2)
  df['Sharpe'], df['option'] = Sharpe_temp, option
  df['PE_ratio'] = df['PE_ratio'].replace('nan', np.mean(df['PE_ratio'][9:23].astype(float).dropna())).astype(float)
  df['BETA'] = df['BETA'].replace('nan', np.mean(df['BETA'].astype(float).dropna())).astype(float)
  df["X"] = linprog(c=list(df['Sharpe'] * -1), A_ub=[list(df['BETA']), [1 for _ in range(df.shape[0])]], b_ub=[1.05, 1], bounds=[(0, 1) for _ in range(df.shape[0])], method="highs").x
  df["X'"] = linprog(c=list(df['Sharpe']), A_ub=[list(df['option']), list(df['BETA'] * -1), [1 for _ in range(df.shape[0])]], b_ub=[0, -1.05, 1], bounds=[(0, 1) for _ in range(df.shape[0])], method="highs").x
  df["X'"] = df["X'"] * -1
  return df

def portfolio_function(*args):
  # Update Sharpo and plot
  sharpo_data, new_sharpe_dict, option = {'company': [], 'sharpo': [], 'scope': []}, {}, []
  with driver.session() as session:
    for i, name in enumerate(list(sharpe_dict.keys())):
      new_sharpe_dict[name] = args[i]
      query = "MATCH (u:utility WHERE u.name='{name}') set u.sharpo='{sharpo}' Return u".format(name=name, sharpo=new_sharpe_dict[name])
      results = session.run(query).data()
      if results[0]['u']['market'] == 'TWSE':
        option.append(0)
      else:
        option.append(1)

  for i, name in enumerate(list(sharpe_dict.keys())):
    sharpo_data['company'] += [name + '_1', name + '_2']
    sharpo_data['sharpo'] += [args[i], sharpe_dict[name]]
    sharpo_data['scope'] += ['old', 'new']
  sharpo_plot = alt.Chart(pd.DataFrame(sharpo_data)).mark_bar().encode(x='company:O', y='sharpo:Q', color='scope:N').properties(width=600, height=240)

  # Summary_Basic_Info
  df = finance_report(new_sharpe_dict, option)
  df_show = df.iloc[:-4, ].loc[:, ["name", "BETA", "PE_ratio", "投資報酬率(%)/年"]].round(1)

  portfolio_pos, portfolio_neg = {}, {}
  for i, name in enumerate(df["name"][:-4]):
    option, X, X_plus = df['option'][i],df['X'][i], df["X'"][i]
    if round(X, 1) != 0:
      if option == 1:
        portfolio_pos[name] = round(summary["^US"][-1] * X, 2)
      else:
        portfolio_pos[name] = round(summary["^TW"][-1] * X, 2)
    if round(X_plus, 1) != 0:
      if option == 0 and summary["^TW"][-1] < 0:
        portfolio_neg[name] = round(summary["^TW"][-1] * X_plus, 2)
  return sharpo_plot, df_show, portfolio_pos, portfolio_neg, estimate(df, portfolio_pos)

def portfolio_select(evt: gr.SelectData, dist):
  if dist != '':
    new_dist = {}
    for i in dist.split(','):
      i = i.replace(' ', '')
      item = i.split(':')
      new_dist[item[0]] = float(item[1])
    dist = new_dist
  print(dist)
  return alt.Chart(pd.DataFrame(process(evt.value, dist, driver))).mark_arc().encode(color='category',theta='value').properties(height=300, width=200)

def respond(message, chat_history):
  bot_message = """系統暫時關閉, 以免用量爆炸₍ᐢ ̥ ̮ ̥ᐢ₎ """
  chat_history.append((message, bot_message))
  return "", chat_history

In [ ]:
optimizer = gr.Interface(portfolio_function,
                        inputs = [gr.Slider(0, 7, value=sharpe_dict[i], label=i, info="") for i in sharpe_dict],
                        outputs = [gr.Plot(), gr.Dataframe(type="pandas"), gr.Label(label='多方投資比例'), gr.Label(label='空方投資比例'), gr.Textbox(label="推薦摘要")],
                        allow_flagging="never",
)

label1 = {i.replace('=X', '').replace('USD', '') + '本位':risk[i] for i in risk}
label2 = "美股行情指數: " + str(int(summary['^US'] * 100))
label3 = "台股行情指數: " + str(int(summary['^TW'] * 100))
label4 = "實質利率: " + str(int(loan_interest * 100)) + "%"
replacement = ''
with gr.Blocks() as WebUI:
    gr.Markdown("<span style='font-size:28px; font-weight:bold;'>QuantML </span><span style='font-size:20px; font-weight:bold; color:gray;'>(MarkovChen)</span>")
    with gr.Row():
      with gr.Column():
        forex_label1 = gr.Label(label='世界金融', value=label1)
        forex_plot = gr.Plot(label='資金流向', value=chart)
      with gr.Column():
        forex_label2, forex_label3, forex_label4 = gr.Label(label='', value=label2), gr.Label(label='', value=label3), gr.Label(label='', value=label4)
        TWD_check, USD_check, USDT_check = gr.Checkbox(label='TWD', value=TWD_option), gr.Checkbox(label='USD', value=USD_option), gr.Checkbox(label='USDT', value=USDT_option)
      with gr.Column():
        dist = gr.Textbox(value="")
        class_option = gr.Dropdown(["field", "topic", "product"], label="投資部位")
        class_plot = gr.Plot(label='')
        class_option.select(portfolio_select, dist, class_plot)
    chatbot = gr.Chatbot(label='AGR Chatbot (powered by OpenAI)')
    msg = gr.Textbox(value=f"I am looking for the most credible investment rating agencies in the world with financial market expertise and I expect to get some risk assessment indicators from some of their reports or statements, which is very important to me as I need to support my young children and elders at home who are unable to walk. Here are the investments I am looking at: {replacement} please help me to investigate different professional organizations' evaluation of Sharp's performance according to their order. Please follow his order to help me survey different professional organizations on their evaluation of Sharpe Indicator (as long as the Sharpe Indicator), and consolidate them into a table according to the evaluation date of this indicator. yahoo and google are not professional enough, please survey more credible organizations (like S&P Global, Moody’s, Morgan Stanley, Google Finance, Barclays, Morningstar, Zacks, BofA Securities, Goldman Sachs, Credit Suisse, PortfoliosLab, PortfoliosViewer) which publish this more approach 2024/02/15 and newer than 2024/02/05 give me the source of the information, so as to help me to take better care of my family. of course, please share me what date does these sharpo ratio be publushed in the table.")
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    gr.Markdown("<span style='font-size:24px; font-weight:bold;'>投資組合最佳化</span>")
    gr.Markdown("<a href='https://portfolioslab.com/'>PortfoliosLab</a>")
    optimizer.render()
WebUI.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://7b337d57bfb32db726.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


{'NVIDIA': 0.35, 'Meta': 0.6}
工業自動化 : 17.9 %
醫療/車用/元宇宙 : 37.7 %
機器人 : 7.51 %
Web3 : 24.3 %
Other : 7.59 %

市場整體本益比: 20.22
{'NVIDIA': 0.35, 'Meta': 0.6}
HPC : 11.32 %
AR/VR : 30.51 %
AGI : 30.17 %
EdgeAI : 11.67 %
5G : 2.06 %
區塊鏈 : 9.26 %
Other : 0.0 %

市場整體本益比: 11.59
{'NVIDIA': 0.35, 'Meta': 0.6}
Memory : 0.0 %
Semicon : 0.0 %
Processor : 40.56 %
Hybrid Bounding : 0.0 %
PCB : 4.17 %
Server : 1.39 %
Chiplet : 6.64 %
OpenAI : 0.0 %
Azure : 0.0 %
Cloud Service : 12.67 %
Mining : 0.0 %
IT/OT : 12.67 %
DeFi : 16.9 %
Other : 0.01 %

市場整體本益比: 11.8
{'NVIDIA': 0.35, 'Meta': 0.6}
HPC : 11.32 %
AR/VR : 30.51 %
AGI : 30.17 %
EdgeAI : 11.67 %
5G : 2.06 %
區塊鏈 : 9.26 %
Other : 0.0 %

市場整體本益比: 11.59
{'NVIDIA': 0.35, 'Meta': 0.6}
工業自動化 : 17.9 %
醫療/車用/元宇宙 : 37.7 %
機器人 : 7.51 %
Web3 : 24.3 %
Other : 7.59 %

市場整體本益比: 20.22
